In [53]:
import mrob
import numpy as np
import math

In [54]:
graph = mrob.FGraph()

In [55]:
graph.add_factor_1pose_3d?

Docstring: add_factor_1pose_3d(self: mrob.mrob.FGraph, arg0: mrob.mrob.geometry.SE3, arg1: int, arg2: numpy.ndarray[numpy.float64[6, 6]]) -> int
Type:      method

In [56]:
# graph.add_node_landmark_3d()
# point description - params by descriptor (by image)
#  
# image -> feature -> 3d_feature_coords -> descriptor -> add/update to mrob
# possible descriptors: orb/nn/
#
# descriptor ->desciption 
# 
#
#
# optimize only robot_trajectory
#
# how to optimize feature comparison (k-neighbours etc)
#   
# 

In [57]:
import cv2
from matplotlib import pyplot as plt
def show_img(img):
    plt.figure(figsize=(14,8))
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.plot()

In [58]:
img_1 = cv2.imread('datasets/rgbd_dataset_freiburg1_xyz/rgb/1305031102.175304.png')
depth_1 = cv2.imread('datasets/rgbd_dataset_freiburg1_xyz/depth/1305031102.160407.png', cv2.IMREAD_GRAYSCALE)
img_2 = cv2.imread('datasets/rgbd_dataset_freiburg1_xyz/rgb/1305031102.211214.png')

In [59]:
def match_images(img_1, img_2):
    img1 = img_1.copy()
    img2 = img_2.copy()
    # orb = cv2.ORB_create()
    sift = cv2.SIFT_create() 
    kp_1, des_1 = sift.detectAndCompute(cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY), None)
    kp_2, des_2 = sift.detectAndCompute(cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY), None)
    print(des_1.shape)
    print(des_2.shape)
    bf = cv2.BFMatcher()  
    # matches = bf.match(des_1, des_2)
    matches = bf.knnMatch(des_1, des_2, k=2)
    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.3*n.distance:
            good.append([m])


    # print(matches)
    img1 = cv2.drawKeypoints(img1, kp_1, None, color=(0,255,0), flags=0)
    # show_img(img1)
    img2 = cv2.drawKeypoints(img2, kp_2, None, color=(0,255,0), flags=0)
    # show_img(img2)
    img3 = cv2.drawMatchesKnn(img1,kp_1,img2,kp_2,good, None, flags=2)
    show_img(img3)
    return kp_1,kp_2,good, des_1, des_2

In [60]:
def match_images_v2(img_1, img_2):
    img1 = img_1.copy()
    img2 = img_2.copy()
    # orb = cv2.ORB_create()
    orb = cv2.ORB_create()
    kp_1, des_1 = orb.detectAndCompute(img1, None)
    kp_2, des_2 = orb.detectAndCompute(img2, None)
    print(des_1.shape)
    print(des_2.shape)
    bf = cv2.BFMatcher()  
    # matches = bf.match(des_1, des_2)
    matches = bf.knnMatch(des_1, des_2, k=2)
    
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append([m])


    # print(matches)
    img1 = cv2.drawKeypoints(img1, kp_1, None, color=(0,255,0), flags=0)
    # show_img(img1)
    img2 = cv2.drawKeypoints(img2, kp_2, None, color=(0,255,0), flags=0)
    # show_img(img2)
    img3 = cv2.drawMatchesKnn(img1,kp_1,img2,kp_2,good, None, flags=2)
    show_img(img3)
    return kp_1,kp_2,good, des_1, des_2

In [61]:
kp1, kp2, matched, des1, des2 = match_images_v2(img_1,img_2)

(500, 32)
(500, 32)


<IPython.core.display.Javascript object>

In [62]:
a, b = map(int, (kp1[0].pt))

In [63]:
a, b

(264, 284)

In [64]:
p1 = []
p2 = []

for match in matched:
    p1.append(match[0].queryIdx)
    p2.append(kp2[match[0].trainIdx].pt)

In [65]:
kp1

(< cv2.KeyPoint 0x7fb674391440>,
 < cv2.KeyPoint 0x7fb6741ba8e0>,
 < cv2.KeyPoint 0x7fb6741baa90>,
 < cv2.KeyPoint 0x7fb6741ba940>,
 < cv2.KeyPoint 0x7fb79c4b37b0>,
 < cv2.KeyPoint 0x7fb67418f330>,
 < cv2.KeyPoint 0x7fb67418fe70>,
 < cv2.KeyPoint 0x7fb67418e580>,
 < cv2.KeyPoint 0x7fb66f72f300>,
 < cv2.KeyPoint 0x7fb66f72fa80>,
 < cv2.KeyPoint 0x7fb66f72e520>,
 < cv2.KeyPoint 0x7fb67fa8b810>,
 < cv2.KeyPoint 0x7fb67fa8b7e0>,
 < cv2.KeyPoint 0x7fb67fa8a9a0>,
 < cv2.KeyPoint 0x7fb67fa8a910>,
 < cv2.KeyPoint 0x7fb67fa8b930>,
 < cv2.KeyPoint 0x7fb67fa8b8d0>,
 < cv2.KeyPoint 0x7fb6741402a0>,
 < cv2.KeyPoint 0x7fb6740fda70>,
 < cv2.KeyPoint 0x7fb6741589c0>,
 < cv2.KeyPoint 0x7fb6741583c0>,
 < cv2.KeyPoint 0x7fb67415a6a0>,
 < cv2.KeyPoint 0x7fb67415ad00>,
 < cv2.KeyPoint 0x7fb67415a2b0>,
 < cv2.KeyPoint 0x7fb67415b9f0>,
 < cv2.KeyPoint 0x7fb67415a130>,
 < cv2.KeyPoint 0x7fb67415aeb0>,
 < cv2.KeyPoint 0x7fb67415bbd0>,
 < cv2.KeyPoint 0x7fb67415bf30>,
 < cv2.KeyPoint 0x7fb67415b990>,
 < cv2.Key

In [66]:
kp2

(< cv2.KeyPoint 0x7fb66d1058c0>,
 < cv2.KeyPoint 0x7fb66d1058f0>,
 < cv2.KeyPoint 0x7fb66d105920>,
 < cv2.KeyPoint 0x7fb66d105950>,
 < cv2.KeyPoint 0x7fb66d105980>,
 < cv2.KeyPoint 0x7fb66d1059b0>,
 < cv2.KeyPoint 0x7fb66d1059e0>,
 < cv2.KeyPoint 0x7fb66d105a10>,
 < cv2.KeyPoint 0x7fb66d105a40>,
 < cv2.KeyPoint 0x7fb66d105a70>,
 < cv2.KeyPoint 0x7fb66d105aa0>,
 < cv2.KeyPoint 0x7fb66d105ad0>,
 < cv2.KeyPoint 0x7fb66d105b00>,
 < cv2.KeyPoint 0x7fb66d105b30>,
 < cv2.KeyPoint 0x7fb66d105b60>,
 < cv2.KeyPoint 0x7fb66d105b90>,
 < cv2.KeyPoint 0x7fb66d105bc0>,
 < cv2.KeyPoint 0x7fb66d105bf0>,
 < cv2.KeyPoint 0x7fb66d105c20>,
 < cv2.KeyPoint 0x7fb66d105c50>,
 < cv2.KeyPoint 0x7fb66d105c80>,
 < cv2.KeyPoint 0x7fb66d105cb0>,
 < cv2.KeyPoint 0x7fb66d105ce0>,
 < cv2.KeyPoint 0x7fb66d105d10>,
 < cv2.KeyPoint 0x7fb66d105d40>,
 < cv2.KeyPoint 0x7fb66d105d70>,
 < cv2.KeyPoint 0x7fb66d105da0>,
 < cv2.KeyPoint 0x7fb66d105dd0>,
 < cv2.KeyPoint 0x7fb66d105e00>,
 < cv2.KeyPoint 0x7fb66d105e30>,
 < cv2.Key

In [67]:
len(p2)

163

In [68]:
len(kp2)

500

In [69]:
def get_pixel_coordinates(img, depth):
    pass

In [70]:

depth_instensity = np.array(256 * depth_1 / 0x0fff,
                            dtype=np.uint8)

In [71]:
print(depth_1.max())
# depth_1 *=3

72


In [72]:
plt.imshow(depth_1)

<IPython.core.display.Javascript object>

In [73]:
plt.imshow(depth_1,cmap='gray')

<IPython.core.display.Javascript object>

In [74]:
# # Depth camera parameters:
# FX_DEPTH = cam_params[0,0]
# FY_DEPTH = cam_params[1,1]
# CX_DEPTH = cam_params[0,2]
# CY_DEPTH = cam_params[1,2]

FX_DEPTH = 525.0  # focal length x
FY_DEPTH = 525.0  # focal length y
CX_DEPTH = 319.5  # optical center x
CY_DEPTH = 239.5  # optical center y

In [81]:
# compute point cloud:
factor = 5000 # for the 16-bit PNG files
pcd = []
height, width = depth_1.shape
for i in range(height):
   for j in range(width):
       if(depth_1[i][j] == 0):
           continue
       z = depth_1[i][j]/factor
       x = (j - CX_DEPTH) * z / FX_DEPTH
       y = (i - CY_DEPTH) * z / FY_DEPTH
       pcd.append([x, -y, -z])

In [76]:
import open3d as o3d

In [82]:
pcd_o3d = o3d.geometry.PointCloud()  # create point cloud object
pcd_o3d.points = o3d.utility.Vector3dVector(pcd)  # set pcd_np as the point cloud points
# Visualize:
o3d.visualization.draw_geometries([pcd_o3d])

In [83]:
pcd = np.asarray(pcd)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pcd[:,0],pcd[:,1], pcd[:,2],c='r', marker='o')
plt.show()

<IPython.core.display.Javascript object>

In [79]:
a

264

In [80]:
np.asarray(list(a.keys()))

AttributeError: 'int' object has no attribute 'keys'

In [ ]:
bf = cv2.BFMatcher()

In [ ]:
des1

array([[  9, 255, 122, ..., 136, 135, 251],
       [213, 204, 142, ..., 120, 248,  14],
       [232, 185,  58, ...,   0, 198, 187],
       ...,
       [131,  94, 123, ..., 208,  33, 121],
       [189,  88,  87, ..., 179, 102, 188],
       [138,  28, 182, ...,  68, 173, 222]], dtype=uint8)

In [ ]:
des1

array([[  9, 255, 122, ..., 136, 135, 251],
       [213, 204, 142, ..., 120, 248,  14],
       [232, 185,  58, ...,   0, 198, 187],
       ...,
       [131,  94, 123, ..., 208,  33, 121],
       [189,  88,  87, ..., 179, 102, 188],
       [138,  28, 182, ...,  68, 173, 222]], dtype=uint8)

In [ ]:
des2 

array([[105, 139, 123, ..., 138, 112,  41],
       [  6, 141,  14, ...,  90, 177, 128],
       [ 16, 207, 107, ..., 128, 180, 232],
       ...,
       [ 33,  79, 199, ..., 179, 120, 188],
       [202, 188, 182, ...,  68, 175, 222],
       [ 41,  77, 183, ..., 168, 208,  40]], dtype=uint8)

In [ ]:
a = np.array([2, 3, 4, 6, 8])

In [ ]:
a

[2, 3, 4, 6, 8]

In [ ]:
idx = [(0), (2), (3)]

In [ ]:
a[idx]

array([2, 4, 6])